# Generación de direcciones en Bitcoin

In [ ]:
import os
import ecdsa
import hashlib

# 1. Generar clave privada (32 bytes aleatorios)
private_key = os.urandom(32)
print("Clave privada (hex):", private_key.hex())

Clave privada (hex): 6621cc98ad248716c7901c6b6e03c9f857f44bf503cf68378c8f83f53f4d6c21


In [18]:
# 2. Crear objeto de clave privada con curva secp256k1
sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)

In [19]:
# 3. Derivar clave pública (sin comprimir)
vk = sk.get_verifying_key()
public_key_bytes = b'\04' + vk.to_string()  # 0x04 = indicador de clave sin comprimir
print("Clave pública (hex):", public_key_bytes.hex())

Clave pública (hex): 0422130d0b7fb3725bde1c4944ad9e1e5ba0d680879664f34b622c573dcd213c1cdce4aee9fd7b66722bfb581da8d7386a098cfa592b2443181e89ff9177e0116c


In [20]:
# 4. Hash de la clave pública (SHA-256 → RIPEMD160)
sha256_hash = hashlib.sha256(public_key_bytes).digest()
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256_hash)
public_key_hash = ripemd160.digest()

In [22]:
# 5. Agregar versión (0x00 para mainnet)
versioned_payload = b'\x00' + public_key_hash

# 6. Checksum (SHA256 doble, primeros 4 bytes)
checksum = hashlib.sha256(hashlib.sha256(versioned_payload).digest()).digest()[:4]

ALPHABET = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"
def b58encode(b: bytes) -> str:
    n = int.from_bytes(b, "big")
    s = ""
    while n > 0:
        n, r = divmod(n, 58)
        s = ALPHABET[r] + s
    pad = 0
    for c in b:
        if c == 0: pad += 1
        else: break
    return ("1" * pad) + s
    
# 7. Dirección final en Base58Check
address = b58encode(versioned_payload + checksum)
print("Dirección Bitcoin:", address)

Dirección Bitcoin: 1HxsZLXVtDtxyrnqbmVzEj3sjtdELg6fhS
